In [2]:
pip install ortools

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

# Load LSTM output
df = pd.read_csv('predicted_demand_output.csv')

# Use predicted demand ONLY
demand = df['Predicted_Demand'].values

print("Sample predicted demand:", demand[:10])
print("Total time slots:", len(demand))


Sample predicted demand: [228.79657 296.53146 362.09537 424.80084 485.36276 544.14825 601.3499
 330.4563  433.7373  294.01358]
Total time slots: 3503


In [3]:
# Select a smaller window for optimization (e.g., 20 demand points)
NUM_ZONES = 20
zones_demand = np.round(demand[:NUM_ZONES]).astype(int)

print("Zones demand:", zones_demand)


Zones demand: [229 297 362 425 485 544 601 330 434 294 304 308 313 316 319 321 322 186
 185 181]


In [4]:
# --- FIX: Split large demand into capacity-sized units ---
vehicle_capacity = 300

expanded_demand = []

for d in zones_demand:
    while d > vehicle_capacity:
        expanded_demand.append(vehicle_capacity)
        d -= vehicle_capacity
    if d > 0:
        expanded_demand.append(d)

zones_demand = expanded_demand

print("Expanded demand units:", zones_demand)
print("Total demand units:", len(zones_demand))


Expanded demand units: [229, 297, 300, 62, 300, 125, 300, 185, 300, 244, 300, 300, 1, 300, 30, 300, 134, 294, 300, 4, 300, 8, 300, 13, 300, 16, 300, 19, 300, 21, 300, 22, 186, 185, 181]
Total demand units: 35


In [12]:
# Capacity-based optimization (stable fallback)

vehicle_capacity = 300
zones = zones_demand.copy()

vehicles = []
current_vehicle = []
current_load = 0

for i, demand in enumerate(zones):
    if current_load + demand <= vehicle_capacity:
        current_vehicle.append(i)
        current_load += demand
    else:
        vehicles.append((current_vehicle, current_load))
        current_vehicle = [i]
        current_load = demand

# Append last vehicle
vehicles.append((current_vehicle, current_load))

print("Optimized vehicle allocation:\n")
for idx, (zones, load) in enumerate(vehicles):
    print(f"Vehicle {idx}: Zones {zones}, Load = {load}")

# Sort demand units in descending order for better packing
zones_demand = sorted(zones_demand, reverse=True)

print("\nTotal vehicles used:", len(vehicles))


Optimized vehicle allocation:

Vehicle 0: Zones [0], Load = 300
Vehicle 1: Zones [1], Load = 300
Vehicle 2: Zones [2], Load = 300
Vehicle 3: Zones [3], Load = 300
Vehicle 4: Zones [4], Load = 300
Vehicle 5: Zones [5], Load = 300
Vehicle 6: Zones [6], Load = 300
Vehicle 7: Zones [7], Load = 300
Vehicle 8: Zones [8], Load = 300
Vehicle 9: Zones [9], Load = 300
Vehicle 10: Zones [10], Load = 300
Vehicle 11: Zones [11], Load = 300
Vehicle 12: Zones [12], Load = 300
Vehicle 13: Zones [13], Load = 300
Vehicle 14: Zones [14], Load = 300
Vehicle 15: Zones [15], Load = 297
Vehicle 16: Zones [16], Load = 294
Vehicle 17: Zones [17], Load = 244
Vehicle 18: Zones [18], Load = 229
Vehicle 19: Zones [19], Load = 186
Vehicle 20: Zones [20], Load = 185
Vehicle 21: Zones [21], Load = 185
Vehicle 22: Zones [22], Load = 181
Vehicle 23: Zones [23, 24], Load = 259
Vehicle 24: Zones [25, 26, 27, 28, 29, 30, 31, 32, 33, 34], Load = 196

Total vehicles used: 25


In [13]:
baseline_vehicles = len(zones_demand)
optimized_vehicles = len(vehicles)

print("\n--- Comparison ---")
print("Vehicles without optimization:", baseline_vehicles)
print("Vehicles with optimization:", optimized_vehicles)
print("Reduction:", baseline_vehicles - optimized_vehicles)



--- Comparison ---
Vehicles without optimization: 35
Vehicles with optimization: 25
Reduction: 10


In [8]:
import pandas as pd

# Prepare results for dashboard
results = []

for vehicle_id, (zones, load) in enumerate(vehicles):
    results.append({
        "Vehicle_ID": vehicle_id,
        "Assigned_Zones": ", ".join(map(str, zones)),
        "Total_Load": load,
        "Capacity_Used_Percent": round((load / 300) * 100, 2)
    })

# Create DataFrame
opt_df = pd.DataFrame(results)

# Save to CSV
opt_df.to_csv("vehicle_allocation_results2.csv", index=False)

print("Optimization results saved to vehicle_allocation_results.csv")


Optimization results saved to vehicle_allocation_results.csv
